In [ ]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init("/home/pyspark/spark")

In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .enableHiveSupport() \
    .getOrCreate()


# spark.xxxxxと記載することで処理を分散させることが可能です。

# 今回利用するデータの確認

今回は、datafileフォルダ配下の

- jinko.csv(各年代の都道府県ごとの人口)
- kenmei_master.csv(都道府県コードをまとめている)

を利用していきます。

In [ ]:
#　データの読み込みを行う

from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col
import pyspark.sql.functions as F

struct = StructType([
    StructField("code", StringType(), False),
    StructField("gengo", StringType(), False),
    StructField("wareki", StringType(), False),
    StructField("seireki", StringType(), False),
    StructField("chu", StringType(), False),
    StructField("sokei", StringType(), False),
    StructField("jinko_male", StringType(), False),
    StructField("jinko_female", StringType(), False)
])
df=spark.read.option("multiLine", "true").option("encoding", "UTF-8") \
    .csv("./datafile/jinko.csv", header=False, sep=',', inferSchema=False,schema=struct)

struct2 = StructType([
    StructField("code", StringType(), False),
    StructField("kenmei", StringType(), False)
])
df2=spark.read.option("multiLine", "true").option("encoding", "UTF-8") \
    .csv("./datafile/kenmei_master.csv", header=False, sep=',', inferSchema=False,schema=struct2)


struct3 = StructType([
    StructField("codes", StringType(), False),
    StructField("kenmei", StringType(), False)
])
df3=spark.read.option("multiLine", "true").option("encoding", "UTF-8") \
    .csv("./datafile/kenmei_master.csv", header=False, sep=',', inferSchema=False,schema=struct3)


# タグクオリティの算出


In [ ]:
# タグクオリティ

# 今までのテスト結果をデータフレームに保存してみます

In [ ]:
# 結果の結合をおこなってdf_resultデータフレームに保存します


In [ ]:
# mysqlへのデータ書き込み
# mysqlのテーブルは勝手に作ってくれます。

df_result.write.format('jdbc').options(
      url='jdbc:mysql://db_mysql/metadata?enabledTLSProtocols=TLSv1.2',
      driver='com.mysql.jdbc.Driver',
      dbtable='metadatas',
      user='root',
      password='root').mode('overwrite').save()

# mysqlでの確認をしてみましょう

```
mysql -h db_mysql -u root -proot  metadata
select * from metadatas
```

In [ ]:
# 最後はSparkをクローズする
spark.stop()
spark.sparkContext.stop()